# MNIST Digit Classification with our own Framework

Lab Assignment from [AI for Beginners Curriculum](https://github.com/microsoft/ai-for-beginners).

### Reading the Dataset

This code download the dataset from the repository on the internet. You can also manually copy the dataset from `/data` directory of AI Curriculum repo.

In [1]:
!rm *.pkl
!wget https://raw.githubusercontent.com/microsoft/AI-For-Beginners/main/data/mnist.pkl.gz
!gzip -d mnist.pkl.gz

rm: cannot remove '*.pkl': No such file or directory
--2025-07-16 07:16:15--  https://raw.githubusercontent.com/microsoft/AI-For-Beginners/main/data/mnist.pkl.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16168813 (15M) [application/octet-stream]
Saving to: ‘mnist.pkl.gz’

mnist.pkl.gz        100%[===================>]  15.42M  --.-KB/s    in 0.1s    

2025-07-16 07:16:16 (135 MB/s) - ‘mnist.pkl.gz’ saved [16168813/16168813]



In [4]:
with open('mnist.pkl','rb') as f:
    MNIST = pickle.load(f,  encoding='latin1')

In [6]:
labels = MNIST['Train']['Labels']
data = MNIST['Train']['Features']

TypeError: tuple indices must be integers or slices, not str

In [15]:
training_set, validation_set, test_set = MNIST
data, labels = training_set

Let's see what is the shape of data that we have:

In [13]:
data.shape

(50000, 784)

### Splitting the Data

We will use Scikit Learn to split the data between training and test dataset:

In [16]:
from sklearn.model_selection import train_test_split

features_train, features_test, labels_train, labels_test = train_test_split(data,labels,test_size=0.2)

print(f"Train samples: {len(features_train)}, test samples: {len(features_test)}")

Train samples: 40000, test samples: 10000


In [21]:
features_train, labels_train = data, labels
features_test, labels_test = test_set

print(f"Train samples: {len(features_train)}, test samples: {len(features_test)}")

Train samples: 50000, test samples: 10000


### Instructions

1. Take the framework code from the lesson and paste it into this notebook, or (even better) into a separate Python module
1. Define and train one-layered perceptron, observing training and validation accuracy during training
1. Try to understand if overfitting took place, and adjust layer parameters to improve accuracy
1. Repeat previous steps for 2- and 3-layered perceptrons. Try to experiment with different activation functions between layers.
1. Try to answer the following questions:
    - Does the inter-layer activation function affect network performance?
    - Do we need 2- or 3-layered network for this task?
    - Did you experience any problems training the network? Especially as the number of layers increased.
    - How do weights of the network behave during training? You may plot max abs value of weights vs. epoch to understand the relation.

In [ ]:
net = Net()
net.add(PerceptronLayer(784, 10))
loss_fn = CrossEntropyLoss()

def train(epochs=5, lr=0.1, batch_size = 128):

    epochs = epochs
    lr = lr
    batch_size = batch_size
    for epoch in range(epochs):
        idx = np.random.permutation(len(train_data))
        for start in range(0, len(train_data), batch_size):
            end = start + batch_size
            X_batch = train_data[idx[start:end]]
            y_batch = train_labels[idx[start:end]]
            logits = net.forward(X_batch)
            probs = softmax(logits)
            loss = loss_fn.forward(probs, y_batch)
            grad = loss_fn.backward(loss)
            net.backward(grad)
            net.update(lr)

    train_preds = np.argmax(softmax(net.forward(train_data)), axis=1)
    train_acc = (train_preds == train_labels).mean()

    val_preds = np.argmax(softmax(net.forward(val_data)), axis=1)
    val_acc = (val_preds == val_labels).mean()

    print(f"Epoch {epoch+1}: Train Accuracy = {train_acc:.4f}, Val Accuracy = {val_acc:.4f}")